In [1]:
import pandas as pd
from requests.packages import target

positive_edges = pd.read_csv('../../data/edgelist.txt', sep=',', header=None)
positive_edges.columns = ['source', 'target']
tfidf_embeddings = pd.read_csv('../../data/abstracts_bert_embeddings.csv')
tfidf_embeddings.head(2)

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,paper_id
0,-0.468760,0.144223,0.365980,-0.029550,0.459321,0.025451,0.075755,0.145586,0.209982,-0.197201,...,-0.095329,-0.033685,-0.299908,-0.350666,-0.263635,0.057492,-0.231705,-0.104059,0.280761,0
1,-0.461041,0.043328,0.285654,-0.173802,0.318974,0.103022,-0.065286,-0.108539,0.133025,-0.252631,...,0.079062,-0.031208,-0.293463,-0.349541,-0.175517,0.092763,-0.276857,-0.268852,0.250939,1


In [2]:
positive_edges_cosine_similarity_df = pd.DataFrame(columns=['source', 'target', 'cosine_similarity'])

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm


positive_edges_cosine_similarity_df['source'] = positive_edges['source']
positive_edges_cosine_similarity_df['target'] = positive_edges['target']

cosine_similarities = []
for idx, row in tqdm(positive_edges_cosine_similarity_df.iterrows(), total=positive_edges_cosine_similarity_df.shape[0]):
    source_vec = tfidf_embeddings[tfidf_embeddings['paper_id'] == row['source']].iloc[:, 1:-1].values
    target_vec = tfidf_embeddings[tfidf_embeddings['paper_id'] == row['target']].iloc[:, 1:-1].values
    if source_vec.size > 0 and target_vec.size > 0:
        sim = cosine_similarity(source_vec, target_vec)[0][0]
    else:
        sim = None
    cosine_similarities.append(sim)

positive_edges_cosine_similarity_df['cosine_similarity'] = cosine_similarities

positive_edges_cosine_similarity_df.to_csv('../../data/training/positive/positive_edges_cosine_similarity.csv', index=False)

  0%|          | 0/1091955 [00:00<?, ?it/s]